In [ ]:
!pip install mlxtend

Our first step is to download a piece of text from Wikipedia and to parse paragraphs.

In [ ]:
from bs4 import BeautifulSoup
import requests

respond = requests.get("https://en.wikipedia.org/wiki/Poznań")
soup = BeautifulSoup(respond.text, "lxml")
page = soup.find_all('p')

raw_text = [paragraph.text for paragraph in page]

print(raw_text)

Next, we will split the text into paragraphs and remove the lines with less than 3 words.

In [ ]:
text = [ line.split() for line in raw_text if len(line) > 2 ]

for line in text[:10]:
    print(line)

Our text still contains a lot of stop-words and some additional tokens such as 1.2, [2], etc. We will use the `nltk` library to remove the stop-words and we'll transform everything to alpha tokens.

In [ ]:
import nltk

nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

stopwords.words('english')[:10]

In [ ]:
clean_text = [
    [ 
        word.lower() 
        for word 
        in line 
        if word.isalpha() 
        and word.lower() not in stopwords.words('english') 
    ]
    for line 
    in text
]

for line in clean_text[:10]:
    print(line)

Now we are ready to transform the list of lists into the format suitable for association rule mining, i.e., to transform the input lists into boolean flags.

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

te = TransactionEncoder()
te_array = te.fit(clean_text).transform(clean_text)

In [ ]:
# te_array contains binary version of the input data

te_array

In [ ]:
te_array.shape

In [ ]:
# original tokens are preserved in the columns_ field

te.columns_[:10]

`mlxtend` package assumes that the input data are stored as a `pandas.DataFrame`

In [ ]:
df = pd.DataFrame(te_array, columns=te.columns_)

df.head()

Now we are ready to find frequent collections of words.

In [ ]:
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)

frequent_itemsets

We can also mine association rules which will have additional measures of quality and interestingness

In [ ]:
from mlxtend.frequent_patterns import association_rules

?association_rules

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(frequent_itemsets, 
                  metric='confidence', 
                  min_threshold=0.7)

In [ ]:
association_rules(frequent_itemsets, metric='lift', min_threshold=5.0)

Both frequent itemsets and association rules (antecedens and consequents) are returned as `frozenset`s, so we can use [standard API calls](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset) to find subsets, supersets, etc.

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

capital_idx =  rules['antecedents'].apply(lambda x: x.issuperset({'polish'}))
rules[capital_idx]